In [1]:
using JuMP
using DataFrames
using CSV
using DataFrames
using Random
#using LShaped
include("../../LShaped.jl/src/structs.jl")
include("../../LShaped.jl/src/setup.jl")
include("../../LShaped.jl/src/algorithm.jl")

## make sure these files and their dependencies are in their proper place ##
include("../parameters.jl")
include("../get_functions.jl")
include("../modification_functions.jl")
##

construct_ef_model (generic function with 1 method)

In [2]:
## upload these to cluster ##
loaddf = DataFrame(CSV.File("../four_wind_turbine_experiments/loaddata.csv"))
wind103df = DataFrame(CSV.File("../four_wind_turbine_experiments/winddata103.csv"))
wind109df = DataFrame(CSV.File("../four_wind_turbine_experiments/winddata109.csv"))
wind117df = DataFrame(CSV.File("../four_wind_turbine_experiments/winddata117.csv"))
wind122df = DataFrame(CSV.File("../four_wind_turbine_experiments/winddata122.csv"))
##

,scenid,date,h1,h2,h3,h4,h5,h6,h7
,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1/1/2002,77.631,130.172,170.516,144.72,32.067,47.563,45.309
2,2,1/2/2002,36.698,26.988,10.85,2.57,0.018,0.004,0.149
3,3,1/3/2002,5.438,37.495,67.072,72.769,95.062,145.275,197.927
4,4,1/4/2002,406.631,406.637,406.64,406.639,406.64,406.64,406.636
5,5,1/5/2002,404.66,307.087,258.291,308.667,404.333,404.224,401.801
6,6,1/6/2002,67.224,108.341,145.046,118.118,127.979,150.923,126.779
7,7,1/7/2002,27.691,4.203,6.271,12.997,26.301,34.522,42.752
8,8,1/8/2002,186.944,113.873,80.828,57.006,48.256,36.013,20.416
9,9,1/9/2002,306.276,305.284,293.739,290.443,283.613,266.227,228.05


In [3]:
winddfdict = Dict(103 => wind103df, 109 => wind109df, 117 => wind117df, 122 => wind122df)

Dict{Int64, DataFrame} with 4 entries:
  117 => 6940×26 DataFrame…
  122 => 6940×26 DataFrame…
  103 => 6940×26 DataFrame…
  109 => 6940×26 DataFrame…

In [4]:
oldmodel = JuMP.read_from_file("../four_wind_turbine_experiments/four_wind/old_second_stage_model_scen_1.mps")

A JuMP Model
Minimization problem with:
Variables: 4416
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 4944 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3792 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [6]:
for wbus in wind_buses
    wind = (1/wmaxdict[wbus])*(wrtsdict[wbus]/100)*collect(winddfdict[wbus][1,3:26])
    if wbus != 122
        for ts in timesteps
            var = @variable(oldmodel, base_name = string("Pre_{", wbus, "_WIND_1,",ts,"}"))
            @constraint(oldmodel, var >= 0.0, base_name = string("Wlb_{",wbus,",",ts,"}"))
            @constraint(oldmodel, var <= wind[ts], base_name = string("Wub_{",wbus,",",ts,"}"))
        end
    end
end

In [17]:
get_wind_ub(oldmodel, 103, 9)

Wub_{103,9} : Pre_{103_WIND_1,9} <= 5.763912512479202

In [8]:
# to do:
# - attach it to load balance (only at relevent buses at each time step)
# - attach it to ptdf constraint (this is already sorted by bus iirc, no need for hacky fixes)
# thus this will extend into 3/15 at least.

In [18]:
for ts in timesteps 
    con = get_load_balance(oldmodel, 103, ts)
    var = get_wind_variable(oldmodel, 103, ts)
    set_normalized_coefficient(con, var, 1.0)
end

In [20]:
for ts in timesteps 
    con = get_load_balance(oldmodel, 109, ts)
    var = get_wind_variable(oldmodel, 109, ts)
    set_normalized_coefficient(con, var, 1.0)
end

In [21]:
for ts in timesteps 
    con = get_load_balance(oldmodel, 117, ts)
    var = get_wind_variable(oldmodel, 117, ts)
    set_normalized_coefficient(con, var, 1.0)
end

In [25]:
ptdfdf[, Symbol(102)]

38-element Vector{Float64}:
  0.065201445
  0.224136718
  0.042617076
  0.065201445
  0.427259341
 -0.023647495
  0.286793162
  0.013669072
  0.130402889
 -0.015703412
  0.286793162
 -0.027338144
  0.00369065
  ⋮
  0.219733466
 -0.016697507
 -0.015703412
  0.118768156
 -0.00369065
  0.224136718
  3.84e-17
  0.254868093
  0.172191648
  0.146106301
  0.239364281
  0.187469677

In [26]:
ptdfdf

,Line,102,117,121,103,107,110
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,A32-2,0.0652014,0.239593,0.236065,0.121556,0.0499458,0.0435974
2,A5,0.224137,0.0174534,0.0189275,0.0667742,-0.00874511,-0.0517643
3,A21,0.0426171,-0.151468,-0.147542,-0.020102,0.0595956,0.0666609
4,A32-1,0.0652014,0.239593,0.236065,0.121556,0.0499458,0.0435974
5,A18,0.427259,0.384386,0.385253,0.413405,0.43101,0.432571
6,A29,-0.0236475,0.274243,-0.366146,-0.0596783,-0.0138937,-0.00983484
7,A3,0.286793,0.0275746,0.0299035,0.105496,-0.013867,-0.0749224
8,A25-1,0.0136691,-0.158522,-0.288355,0.0344961,0.00803105,0.00568488
9,A28,0.130403,0.479186,0.47213,0.243113,0.0998915,0.0871947


In [29]:
branchtorow = Dict()
count = 0
for br in branches
    for i = 1:38
        if ptdfdf[i,1] == br
            branchtorow[br] = i
            count += 1
            println(count)
        end
    end
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [32]:
for br in branches
    coeff103 = -ptdfdf[branchtorow[br], Symbol(103)]
    coeff109 = -ptdfdf[branchtorow[br], Symbol(109)]
    coeff117 = -ptdfdf[branchtorow[br], Symbol(117)]
    for ts in timesteps
        con = get_ptdf_con(oldmodel, br, ts)
        var103 = get_wind_variable(oldmodel, 103, ts)
        var109 = get_wind_variable(oldmodel, 109, ts)
        var117 = get_wind_variable(oldmodel, 117, ts)
        
        set_normalized_coefficient(con, var103, coeff103)
        set_normalized_coefficient(con, var109, coeff109)
        set_normalized_coefficient(con, var117, coeff117)
    end
end


In [34]:
write_to_file(oldmodel, "../four_wind_turbine_experiments/four_wind/second_stage_model_scen_1.mps")

In [35]:
omodel = JuMP.read_from_file("../four_wind_turbine_experiments/four_wind/old_second_stage_model_scen_1.mps")
nmodel = JuMP.read_from_file("../four_wind_turbine_experiments/four_wind/second_stage_model_scen_1.mps")

A JuMP Model
Minimization problem with:
Variables: 4488
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 5016 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3864 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [36]:
omodel

A JuMP Model
Minimization problem with:
Variables: 4416
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2064 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 4944 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 3792 constraints
`VariableRef`-in-`MathOptInterface.EqualTo{Float64}`: 48 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [39]:
using Gurobi

In [40]:
JuMP.set_optimizer(omodel, Gurobi.Optimizer)
JuMP.set_optimizer(nmodel, Gurobi.Optimizer)

Academic license - for non-commercial use only - expires 2022-08-19
Academic license - for non-commercial use only - expires 2022-08-19


In [41]:
optimize!(omodel)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10800 rows, 4416 columns and 119976 nonzeros
Model fingerprint: 0xa2e55cb1
Coefficient statistics:
  Matrix range     [2e-04, 5e+00]
  Objective range  [5e-02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-02, 7e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 7137 rows and 314 columns
Presolve time: 0.09s
Presolved: 3663 rows, 4263 columns, 84268 nonzeros

Ordering time: 0.09s

Barrier statistics:
 AA' NZ     : 1.462e+05
 Factor NZ  : 1.466e+06 (roughly 15 MBytes of memory)
 Factor Ops : 8.921e+08 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.26 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved

In [42]:
optimize!(nmodel)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10944 rows, 4488 columns and 122856 nonzeros
Model fingerprint: 0x11d4b79e
Coefficient statistics:
  Matrix range     [2e-04, 5e+00]
  Objective range  [5e-02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 7e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 7281 rows and 314 columns
Presolve time: 0.07s
Presolved: 3663 rows, 4335 columns, 86428 nonzeros

Ordering time: 0.10s

Barrier statistics:
 AA' NZ     : 1.462e+05
 Factor NZ  : 1.438e+06 (roughly 15 MBytes of memory)
 Factor Ops : 8.478e+08 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.24 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved